In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize

new_width, new_height = 128, 128  

def preprocess_image(image_path, new_width=128, new_height=128):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    _, binary_image = cv2.threshold(blurred_image, 127, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    resized_closed_image = cv2.resize(closed_image, (new_width, new_height))
    edges = cv2.Canny(resized_closed_image, 100, 200)
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)  
    return edges_colored

def load_dataset(folder_path, label, new_width=128, new_height=128):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"): 
            image_path = os.path.join(folder_path, filename)
            processed_image = preprocess_image(image_path, new_width, new_height)
            images.append(processed_image)
            labels.append(label)
    return images, labels

cheat_path = r"D:\CV_DATASETS\cheat"
non_cheat_path = r"D:\CV_DATASETS\noncheat"

cheat_images, cheat_labels = load_dataset(cheat_path, 1)  
non_cheat_images, non_cheat_labels = load_dataset(non_cheat_path, 0) 

X = np.array(cheat_images + non_cheat_images, dtype='float32')
y = np.array(cheat_labels + non_cheat_labels)

X = normalize(X, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(new_height, new_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1/1 [==============================] - 1s 973ms/step - loss: 0.6943 - accuracy: 0.4167 - val_loss: 0.6339 - val_accuracy: 0.7500
Epoch 2/10
1/1 [==============================] - 0s 107ms/step - loss: 0.3190 - accuracy: 1.0000 - val_loss: 0.7736 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 109ms/step - loss: 0.0505 - accuracy: 1.0000 - val_loss: 1.0860 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 109ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 1.4528 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 109ms/step - loss: 4.8017e-04 - accuracy: 1.0000 - val_loss: 1.8454 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 111ms/step - loss: 4.8224e-05 - accuracy: 1.0000 - val_loss: 2.2411 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 125ms/step - loss: 5.4255e-06 - accuracy: 1.0000 - val_loss: 2.6255 - val_accuracy: 0.5000
Epoch 8/

In [5]:

new_image_path = r"D:\CV_DATASETS\noncheat\Screenshot 2024-02-22 022608.jpg"

new_image_processed = preprocess_image(new_image_path, new_width=128, new_height=128)

new_image_processed_expanded = np.expand_dims(new_image_processed, axis=0)


In [6]:
prediction = model.predict(new_image_processed_expanded)

predicted_class = (prediction > 0.5).astype("int32")

if predicted_class == 0:
    print("The image is classified as non-cheat.")
else:
    print("The image is classified as cheat.")


1/1 [==============================] - 0s 92ms/step
The image is classified as non-cheat.
